In [7]:
import ROOT as r
import math
import array
import os
import sys
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.backends.backend_pdf import PdfPages
import uproot
import numpy as np
from scipy.optimize import curve_fit

#______________Add include directory_______________
current_dir = os.getcwd()
include_dir = os.path.join(current_dir, 'include')
sys.path.insert(0, include_dir)
#__________________________________________________

import CONFIG
import DBPARSE
from UTILITIES import *
from SIMFITS2D import DistributionFits2D
from ROOT import gStyle, TChain, TH1F, TCanvas, TLegend

Welcome to JupyROOT 6.26/06


In [8]:
#available branches
import ROOT as r


def list_branches(rootfile_path, tree_name="Tout"):
    # Open the ROOT file
    root_file = r.TFile.Open(rootfile_path)
    
    # Check if the file was successfully opened
    if not root_file or root_file.IsZombie():
        print(f"Error: Could not open ROOT file '{rootfile_path}'.")
        return
    
    # Access the TTree
    tree = root_file.Get(tree_name)
    
    # Check if the tree exists
    if not tree:
        print(f"Error: Tree '{tree_name}' not found in file '{rootfile_path}'.")
        root_file.Close()
        return
    
    # Get the list of branches
    branch_list = tree.GetListOfBranches()
    
    # Print all branch names
    print(f"Branches in tree '{tree_name}':")
    for branch in branch_list:
        print(branch.GetName())
    
    # Close the ROOT file
    root_file.Close()

# Example usage
rootfile_path = "outfiles/HCal_data_GEN3_sbs100p_nucleon_np_model1.root"
list_branches(rootfile_path)


Branches in tree 'Tout':
runnum
WCut
pCut
nCut
fiduCut
coinCut
ebeam
nu
Q2
W2
dpel
ephi
etheta
pcentral
vz
vx
vy
xtgt
ytgt
thtgt
phtgt
thetabend
xfp
yfp
thfp
phfp
trP
trPx
trPy
trPz
ePS
xPS
eSH
xSH
ySH
eHCAL
xHCAL
yHCAL
xHCAL_exp
yHCAL_exp
dx
dy
ngrinch_hits
xGRINCH
yGRINCH
coin_time
hcal_time
bbcal_time
BPMAx
BPMAy
Rasterx
Rastery
Raster2x
Raster2y
helicity
IHWP
pblkid
tdc
atime
nblk
mag
run
tar
cblkid
cblkatime
cblktime
cblke
nclus
cid
cx
cy
catime
hodoTimes
hodoIDs
hodoTOT
nbars
meanTOT
trigger
triggerID
NdataTriggerID
hodoClusMean


Error in <TList::Clear>: A list is accessing an object (0x36741bb0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x32450110) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x32450540) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x36797d70) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x367980e0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x36798450) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x36798980) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x36798d40) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x367996a0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x36799a

In [9]:
#available branches
import ROOT as r


def list_branches(rootfile_path, tree_name="Tout"):
    # Open the ROOT file
    root_file = r.TFile.Open(rootfile_path)
    
    # Check if the file was successfully opened
    if not root_file or root_file.IsZombie():
        print(f"Error: Could not open ROOT file '{rootfile_path}'.")
        return
    
    # Access the TTree
    tree = root_file.Get(tree_name)
    
    # Check if the tree exists
    if not tree:
        print(f"Error: Tree '{tree_name}' not found in file '{rootfile_path}'.")
        root_file.Close()
        return
    
    # Get the list of branches
    branch_list = tree.GetListOfBranches()
    
    # Print all branch names
    print(f"Branches in tree '{tree_name}':")
    for branch in branch_list:
        print(branch.GetName())
    
    # Close the ROOT file
    root_file.Close()

# Example usage
rootfile_path = "outfiles/Pass1/QE_sim_GEN3_sbs100p_nucleon_np_model2.root"
list_branches(rootfile_path)

Branches in tree 'Tout':
WCut
pCut
nCut
weight
fnucl
fiduCut
coinCut
ebeam
nu
Q2
W2
dpel
ephi
etheta
pcentral
vz
vx
vy
xtgt
ytgt
thtgt
phtgt
xfp
yfp
thfp
phfp
trP
trX
trY
trTh
trPh
ePS
eSH
eHCAL
xHCAL
yHCAL
xHCAL_exp
yHCAL_exp
dx
dy
grinch_track
grinch_clus_size


Error in <TList::Clear>: A list is accessing an object (0x36e6e2a0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x36e6e140) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x36e7c6b0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x36e7c550) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x36e7c460) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x36e6dfe0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x36e5c6c0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x3675cfa0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x36e475d0) already deleted (list name = TList)
Error in <TList::Clear>: A list is accessing an object (0x36e3cb

In [10]:
"""
#inputs hodo
config="3"
prefix="HCal_data"
detector="hodo"
#inputs hcal
config="3"
prefix="HCal_data"
detector="hcal"
"""

'\n#inputs hodo\nconfig="3"\nprefix="HCal_data"\ndetector="hodo"\n#inputs hcal\nconfig="3"\nprefix="HCal_data"\ndetector="hcal"\n'

In [22]:
#define branches
branches=["ebeam","nu","Q2","W2","dpel","ephi","etheta","pcentral","vz","vx"
          ,"vy","xtgt","ytgt","thtgt","phtgt","thetabend","xfp","yfp","thfp"
          ,"phfp","trP","trPx","trPy","trPz","ePS","xPS","eSH","xSH","ySH","eHCAL"
          ,"xHCAL","yHCAL","xHCAL_exp","yHCAL_exp","dx","dy","ngrinch_hits","xGRINCH"
          ,"yGRINCH","coin_time","hcal_time","bbcal_time","BPMAx","BPMAy","Rasterx"
          ,"Rastery","Raster2x","Raster2y","helicity","IHWP","pblkid","tdc","atime"
          ,"nblk","mag","run","tar","cblkid","cblkatime","cblktime","cblke","nclus"
          ,"cid","cx","cy","catime","hodoTimes","hodoIDs","hodoTOT","nbars"]
hodo_branches=["hodoTimes","hodoIDs","hodoTOT","nbars","hodoClusMean"]
hcal_branches=["cblkid","cblkatime","cblktime","cblke","nclus"
          ,"cid","cx","cy","catime"]
trigger_branches=["trigger","triggerID","NdataTriggerID"]
generic_branches=["nu","Q2","W2","vz","vx","vy","dx","dy","eSH","ePS","etheta","trP","coin_time"]

sim_branches=["nu","Q2","W2","vz","vx","vy","dx","dy","eSH","ePS","etheta","trP"]


ml_branches = [
    "ebeam", "nu", "Q2", "W2", "dpel", "ephi", "etheta", "pcentral", "vz", "vx", "vy",
    "xtgt", "ytgt", "thtgt", "phtgt", "xfp", "yfp", "thfp", "phfp", "trP",
    "ePS", "eSH", "eHCAL", "xHCAL", "yHCAL", "xHCAL_exp", "yHCAL_exp",
    "dx", "dy"
]


In [12]:

def ConvertToNP(branches,config,prefix,detector):
    # Open the ROOT file
    with uproot.open(f"outfiles/HCal_data_GEN3_sbs100p_nucleon_np_model1.root") as file:
        # Access the tree (replace 'Tout' with the actual name of your tree if different)
        tree = file["Tout"]

        # Get all branch names
        #branches = tree.keys()

        chunk_size = 2000000  # Define the chunk size
        total_entries = tree.num_entries  # Get the total number of entries
        total_iterations = total_entries // chunk_size + (total_entries % chunk_size > 0)

        for j in range(total_iterations):
            print(f"Processing Chunk {j+1} out of {total_iterations}")

            start = j * chunk_size
            stop = min((j + 1) * chunk_size, total_entries)

            # Read data for this chunk
            data = tree.arrays(branches, entry_start=start, entry_stop=stop, library="np")

            # Save all the branches as a dictionary to .npz
            np.savez(f"/media/research/TOSHIBA EXT/GEn/NumpyData/GEN{config}/{prefix}/{detector}{j}.npz", **data)


In [13]:
def ConvertToNPSim(branches,config,prefix,detector):
    # Open the ROOT file
    with uproot.open(f"outfiles/Pass1/QE_sim_GEN3_sbs100p_nucleon_np_model2.root") as file:
        # Access the tree (replace 'Tout' with the actual name of your tree if different)
        tree = file["Tout"]

        # Get all branch names
        #branches = tree.keys()

        chunk_size = 2000000  # Define the chunk size
        total_entries = tree.num_entries  # Get the total number of entries
        total_iterations = total_entries // chunk_size + (total_entries % chunk_size > 0)
        print(total_entries)
        for j in range(total_iterations):
            print(f"Processing Chunk {j+1} out of {total_iterations}")

            start = j * chunk_size
            stop = min((j + 1) * chunk_size, total_entries)

            # Read data for this chunk
            data = tree.arrays(branches, entry_start=start, entry_stop=stop, library="np")

            # Save all the branches as a dictionary to .npz
            np.savez(f"/media/research/TOSHIBA EXT/GEn/NumpySim/GEN{config}/{detector}{j}.npz", **data)


In [14]:
def ConvertToNPInSim(branches,config,prefix,detector):
    # Open the ROOT file
    with uproot.open(f"outfiles/Pass1/QE_Insim_GEN3_sbs100p_nucleon_np_model2.root") as file:
        # Access the tree (replace 'Tout' with the actual name of your tree if different)
        tree = file["Tout"]

        # Get all branch names
        #branches = tree.keys()

        chunk_size = 2000000  # Define the chunk size
        total_entries = tree.num_entries  # Get the total number of entries
        total_iterations = total_entries // chunk_size + (total_entries % chunk_size > 0)
        print(total_entries)
        for j in range(total_iterations):
            print(f"Processing Chunk {j+1} out of {total_iterations}")

            start = j * chunk_size
            stop = min((j + 1) * chunk_size, total_entries)

            # Read data for this chunk
            data = tree.arrays(branches, entry_start=start, entry_stop=stop, library="np")

            # Save all the branches as a dictionary to .npz
            np.savez(f"/media/research/TOSHIBA EXT/GEn/NumpySim/GEN{config}/{detector}{j}.npz", **data)


In [8]:
#Hodoscope
ConvertToNP(hodo_branches,"3","HCal_data","hodo")

Processing Chunk 1 out of 13
Processing Chunk 2 out of 13
Processing Chunk 3 out of 13
Processing Chunk 4 out of 13
Processing Chunk 5 out of 13
Processing Chunk 6 out of 13
Processing Chunk 7 out of 13
Processing Chunk 8 out of 13
Processing Chunk 9 out of 13
Processing Chunk 10 out of 13
Processing Chunk 11 out of 13
Processing Chunk 12 out of 13
Processing Chunk 13 out of 13


In [16]:
#HCal
ConvertToNP(hcal_branches,"3","HCal_data","hcal")

Processing Chunk 1 out of 13
Processing Chunk 2 out of 13
Processing Chunk 3 out of 13
Processing Chunk 4 out of 13
Processing Chunk 5 out of 13
Processing Chunk 6 out of 13
Processing Chunk 7 out of 13
Processing Chunk 8 out of 13
Processing Chunk 9 out of 13
Processing Chunk 10 out of 13
Processing Chunk 11 out of 13
Processing Chunk 12 out of 13
Processing Chunk 13 out of 13


In [6]:
#Trigger
ConvertToNP(trigger_branches,"3","HCal_data","trigger")

Processing Chunk 1 out of 13
Processing Chunk 2 out of 13
Processing Chunk 3 out of 13
Processing Chunk 4 out of 13
Processing Chunk 5 out of 13
Processing Chunk 6 out of 13
Processing Chunk 7 out of 13
Processing Chunk 8 out of 13
Processing Chunk 9 out of 13
Processing Chunk 10 out of 13
Processing Chunk 11 out of 13
Processing Chunk 12 out of 13
Processing Chunk 13 out of 13


In [25]:
#Generic
ConvertToNP(generic_branches,"3","HCal_data","generic")

Processing Chunk 1 out of 13
Processing Chunk 2 out of 13
Processing Chunk 3 out of 13
Processing Chunk 4 out of 13
Processing Chunk 5 out of 13
Processing Chunk 6 out of 13
Processing Chunk 7 out of 13
Processing Chunk 8 out of 13
Processing Chunk 9 out of 13
Processing Chunk 10 out of 13
Processing Chunk 11 out of 13
Processing Chunk 12 out of 13
Processing Chunk 13 out of 13


In [23]:
#MachineLearning Data/ElasticSim/InelasticSim
ConvertToNP(ml_branches,"3","HCal_data","machinelearningD")
ConvertToNPSim(ml_branches,"3","HCal_data","machinelearningES")
ConvertToNPInSim(ml_branches,"3","HCal_data","machinelearningIS")

Processing Chunk 1 out of 13
Processing Chunk 2 out of 13
Processing Chunk 3 out of 13
Processing Chunk 4 out of 13
Processing Chunk 5 out of 13
Processing Chunk 6 out of 13
Processing Chunk 7 out of 13
Processing Chunk 8 out of 13
Processing Chunk 9 out of 13
Processing Chunk 10 out of 13
Processing Chunk 11 out of 13
Processing Chunk 12 out of 13
Processing Chunk 13 out of 13
264763
Processing Chunk 1 out of 1
74232
Processing Chunk 1 out of 1


In [22]:
#Simulation
ConvertToNPSim(sim_branches,"3","HCal_data","simulation")

264763
Processing Chunk 1 out of 1


In [12]:
#Simulation
ConvertToNPInSim(sim_branches,"3","HCal_data","insimulation")

74232
Processing Chunk 1 out of 1
